In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
from time import sleep


In [2]:
def queryBicipublica(biciID, indicador = 'disponibilidad'):
    try:
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?callback=jQuery19108223016225484279_1505305148763&id=estaciones_de_bicicletas%7C' + biciID
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData.split('(',1)[1][:-1])


        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])

        rotas = 100 - ((bicisDisponibles + posicionesDisponibles)*1. / totales * 100)
        #disponibilidad = bicisDisponibles *1. /  totales * 100
        disponibilidad = bicisDisponibles *1. /  (bicisDisponibles + posicionesDisponibles) * 100
        if disponibilidad > 100.0:
            disponibilidad = 100.0
    except:
        rotas = np.nan
        disponibilidad = np.nan
        
    
    if indicador == 'rotas':
        return rotas
    elif indicador == 'ambas':
        return {'rotas':rotas,'disponibilidad':disponibilidad}
    else:
        return disponibilidad

In [3]:
#base madre
geometria = pd.read_csv('../data/estaciones-de-bicicletas-publicas.csv',sep=';')
#convertir en texto los id de las estaciones para el query
geometria.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in geometria.NRO_EST]
#remover columnas innecesarias
geometria = geometria.loc[:,['X','Y','NOMBRE','NRO_EST']]
geometria.head()



,X,Y,NOMBRE,NRO_EST
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01
1,-58.374822,-34.592589,RETIRO,02
2,-58.368918,-34.611242,ADUANA,03
3,-58.368950,-34.601721,PLAZA ROMA,04
4,-58.369360,-34.627947,PARQUE LEZAMA,06


In [4]:
datetime.datetime.now().hour

0

In [5]:
#modulo tiempo
hora = datetime.datetime.now().hour
#poner un dia limite
diaLimite = 19

fechas = []
disponibilidades = []
roturas = []
biciIDs = []

#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 0:
        print 'hora inicio'
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < diaLimite:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #agrego la disponibilidad de la estacion a la lista
                disponibilidades.append(queryBicipublica(biciID, indicador = 'disponibilidad'))
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'disponibilidad':disponibilidades#,
                      #'roturas':roturas
                      })

hora inicio
fecha: 2017-09-18 00:12:24.031035
2017-09-18 00:12:24.031035
fecha: 2017-09-18 00:28:17.550805
2017-09-18 00:28:17.550805
fecha: 2017-09-18 00:43:59.205273
2017-09-18 00:43:59.205273
fecha: 2017-09-18 00:59:59.354992
2017-09-18 00:59:59.354992
fecha: 2017-09-18 01:15:54.219349
2017-09-18 01:15:54.219349
fecha: 2017-09-18 01:31:41.586183
2017-09-18 01:31:41.586183
fecha: 2017-09-18 01:47:29.306740
2017-09-18 01:47:29.306740
fecha: 2017-09-18 02:03:15.045305
2017-09-18 02:03:15.045305
fecha: 2017-09-18 02:18:56.421359
2017-09-18 02:18:56.421359
fecha: 2017-09-18 02:34:39.379037
2017-09-18 02:34:39.379037
fecha: 2017-09-18 02:50:21.210440
2017-09-18 02:50:21.210440
fecha: 2017-09-18 03:06:13.053771
2017-09-18 03:06:13.053771
fecha: 2017-09-18 03:21:56.385177
2017-09-18 03:21:56.385177
fecha: 2017-09-18 03:37:39.429330
2017-09-18 03:37:39.429330
fecha: 2017-09-18 03:53:22.582158
2017-09-18 03:53:22.582158
fecha: 2017-09-18 04:09:08.558377
2017-09-18 04:09:08.558377
fecha: 2017-

In [6]:
print 'datos', datos.shape
print 'geometria',geometria.shape

datos (16856, 3)
geometria (196, 4)


In [7]:
data = geometria.merge(datos)
print data.shape
data.head()

(16856, 6)


,X,Y,NOMBRE,NRO_EST,disponibilidad,fecha
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01,0.0,2017-09-18 00:12:24.031035
1,-58.392452,-34.583133,FACULTAD DE DERECHO,01,0.0,2017-09-18 00:28:17.550805
2,-58.392452,-34.583133,FACULTAD DE DERECHO,01,0.0,2017-09-18 00:43:59.205273
3,-58.392452,-34.583133,FACULTAD DE DERECHO,01,0.0,2017-09-18 00:59:59.354992
4,-58.392452,-34.583133,FACULTAD DE DERECHO,01,0.0,2017-09-18 01:15:54.219349


In [9]:
pwd

u'/home/pipe/projects/ecobiciStations/src'

In [10]:
data.to_csv('../data/data18sept.csv')

In [11]:
import geopandas as gpd

In [15]:
barrios = gpd.read_file('../data/barrios-rar/barrios_badata.shp')
barrios = barrios.to_crs(epsg=4326)
barrios.head()

,AREA,BARRIO,COMUNA,PERIMETRO,geometry
0,3.118101e+06,CHACARITA,15.0,7725.695228,"POLYGON ((-58.452220824502 -34.59643594220518,..."
1,2.229829e+06,PATERNAL,15.0,7087.513295,POLYGON ((-58.46497711478177 -34.5970050426002...
2,3.613584e+06,VILLA CRESPO,15.0,8132.699348,POLYGON ((-58.4231548125714 -34.59827443880985...
3,3.399596e+06,VILLA DEL PARQUE,11.0,7705.389797,POLYGON ((-58.49400885920002 -34.6153122984289...
4,4.050752e+06,ALMAGRO,5.0,8537.901368,POLYGON ((-58.41227214621448 -34.6145629260479...
